In [19]:
import numpy as np
x = np.array([
    [2.5, 2.4, 0.5],
    [0.5, 0.7, 1.8],
    [2.2, 2.9, 0.4],
    [1.9, 2.2, 1.1],
    [3.1, 3.0, 0.6]
])
y = np.array([0, 1, 0, 1,0])
x_centered=x-x.mean(axis=0)
U,s,Vt=np.linalg.svd(x_centered)
c1=Vt.T[:,0]
c2=Vt.T[:,1]
print(c1)
print(c2)

[-0.66580052 -0.63875032  0.38562638]
[ 0.72455777 -0.4301227   0.53852623]


In [20]:
W2=Vt.T[:,:2]
X2D=x_centered.dot(W2)
print(X2D)

[[-0.55500631  0.05983697]
 [ 2.36378455  0.04201414]
 [-0.71320395 -0.42644433]
 [ 0.20359989  0.03424259]
 [-1.29917417  0.29035063]]


In [21]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)
X2D=pca.fit_transform(x)
print(X2D)

[[ 0.55500631  0.05983697]
 [-2.36378455  0.04201414]
 [ 0.71320395 -0.42644433]
 [-0.20359989  0.03424259]
 [ 1.29917417  0.29035063]]


In [22]:
pca=PCA()
x_train= np.array([
    [2.5, 2.4, 0.5],
    [0.5, 0.7, 1.8],
    [2.2, 2.9, 0.4],
    [1.9, 2.2, 1.1],
    [3.1, 3.0, 0.6]
])
pca.fit(x_train)
cumsum=np.cumsum(pca.explained_variance_ratio_)
d=np.argmax(cumsum>=0.95)+1
print(d)

1


In [23]:
pca=PCA(n_components=2)
x_reduced=pca.fit_transform(x_train)
print(x_reduced)

[[ 0.55500631  0.05983697]
 [-2.36378455  0.04201414]
 [ 0.71320395 -0.42644433]
 [-0.20359989  0.03424259]
 [ 1.29917417  0.29035063]]


In [24]:
x_recovered=pca.inverse_transform(x_reduced)
print(x_recovered)

[[2.45287883 2.56877322 0.6981987 ]
 [0.49663269 0.71206064 1.8141634 ]
 [2.20586801 2.87898264 0.37531828]
 [1.92925382 2.09522199 0.97695402]
 [3.11536664 2.94496151 0.5353656 ]]


In [25]:
rnd_pca=PCA(n_components=2,svd_solver="randomized")
x_reduced=rnd_pca.fit_transform(x_train)
print(x_reduced)

[[ 0.55500631  0.05983697]
 [-2.36378455  0.04201414]
 [ 0.71320395 -0.42644433]
 [-0.20359989  0.03424259]
 [ 1.29917417  0.29035063]]


In [26]:
from sklearn.decomposition import IncrementalPCA
n_components=3
n_batches=2
inc_pca=IncrementalPCA(n_components=n_components)
x_train=np.random.rand(6,4)
for x_batch in np.array_split(x_train,n_batches):
    inc_pca.partial_fit(x_batch)
x_reduced=inc_pca.transform(x_train)
print(x_reduced)

[[-0.25846221 -0.19525157  0.32372065]
 [-0.14214652  0.58714761  0.01934325]
 [ 0.32287681 -0.03124395  0.15064254]
 [-0.3509514  -0.2469905  -0.15975639]
 [ 0.73639492 -0.07525369 -0.11593347]
 [-0.30771161 -0.0384079  -0.21801658]]


In [27]:
from sklearn.decomposition import KernelPCA
rbf_pca=KernelPCA(n_components=2,kernel="rbf",gamma=0.04)
x_reduced=rbf_pca.fit_transform(x_train)
print(x_reduced)

[[-0.07217982 -0.05462247]
 [-0.03903859  0.16485713]
 [ 0.09061802 -0.00904712]
 [-0.09834745 -0.0689681 ]
 [ 0.20535019 -0.02143491]
 [-0.08640235 -0.01078453]]


In [28]:
from ast import Param
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
clf=Pipeline([
    ("kpca",KernelPCA(n_components=2)),
    ("log_reg",LogisticRegression())
])
param_grid=[
    {
        "kpca__gamma":np.linspace(0.03,0.05,10),
        "kpca__kernel":["rbf","sigmoid"]
    }
]
grid_search=GridSearchCV(clf,param_grid,cv=3)
grid_search.fit(x,y)
print(grid_search.best_params_)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


{'kpca__gamma': np.float64(0.03), 'kpca__kernel': 'rbf'}


In [29]:
rbf_pca=KernelPCA(n_components=2,kernel="rbf",gamma=0.0433,fit_inverse_transform=True)
x_reduced=rbf_pca.fit_transform(x)
x_preimage=rbf_pca.inverse_transform(x_reduced)
print(x_preimage)

[[1.72462857 1.89131976 0.72222151]
 [1.61089309 1.77766106 0.76338535]
 [1.72711568 1.89386859 0.71921247]
 [1.70217993 1.87037163 0.73607026]
 [1.73616275 1.90042142 0.70929559]]


In [34]:
from sklearn.manifold import LocallyLinearEmbedding
lle=LocallyLinearEmbedding(n_components=2,n_neighbors=4)
x_reduced=lle.fit_transform(x)
print(x_reduced)

[[ 0.22141777  0.08113412]
 [-0.82187101 -0.08983963]
 [ 0.22147143  0.77530247]
 [-0.08857294 -0.17067589]
 [ 0.46755475 -0.59592107]]
